## Avance 1 
- Benjamin Gonzalez Dominguez
- Fecha: 13-09-2024

In [51]:
import os
import re
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import (
    precision_score,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    classification_report,
)

In [52]:
from urllib.request import urlretrieve
def direct_load(fname):
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df=pd.read_parquet(f'{PATH}/{fname}')
    return df


In [53]:
INSTRUMENT='MATISSE'
RANGE='1m'
REPO_URL='https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'
PATH='sample_data' # Convenient name to be Colab compatible

!mkdir -p $PATH

df_meta = direct_load(f'{INSTRUMENT}-{RANGE}-meta.parket')
df_inst = direct_load(f'{INSTRUMENT}-{RANGE}-traces.parket')
df_subs = direct_load(f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket')
df_tele = direct_load(f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket')

Ya existe el subdirectorio o el archivo -p.
Error mientras se procesaba: -p.
Ya existe el subdirectorio o el archivo sample_data.
Error mientras se procesaba: sample_data.


In [54]:
df_meta.head()

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
0,2019-04-02 01:29:45.681,2019-04-02 01:38:08.541,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,502.0,AT
1,2019-04-02 01:38:08.559,2019-04-02 01:44:02.506,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,353.0,AT
2,2019-04-02 01:45:44.711,2019-04-02 01:58:00.921,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,736.0,AT
3,2019-04-02 01:58:19.863,2019-04-02 02:14:00.315,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,940.0,AT
4,2019-04-02 06:36:28.631,2019-04-02 07:00:16.566,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,1427.0,AT


In [55]:
#Filtro los registros del df_meta en los que tienen error

df_meta_errores = df_meta[(df_meta['ERROR']==True)]
print(f'Hay {len(df_meta_errores)} datos con ERROR=True')
df_meta_errores.head()

Hay 42 datos con ERROR=True


,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,SECONDS,TEL
18,2019-04-02 11:52:49.584,2019-04-02 12:03:38.078,False,MATISSE,bob_ins,MATISSE_gen_tec_check_fringes,True,False,648.0,AT
33,2019-04-03 01:49:30.275,2019-04-03 02:01:22.544,False,MATISSE,bob_ins,MATISSE_img_acq,True,False,712.0,AT
40,2019-04-03 03:02:03.291,2019-04-03 03:10:28.721,False,MATISSE,bob_ins,MATISSE_img_acq,True,False,505.0,AT
62,2019-04-03 13:51:06.769,2019-04-03 13:51:07.795,False,MATISSE,bob_14122,MATISSE_gen_cal_det_h2,True,False,1.0,AT
73,2019-04-03 15:05:25.634,2019-04-03 15:05:26.781,False,MATISSE,bob_100659,MATISSE_gen_cal_det_h2,True,False,1.0,AT


In [56]:
df_inst.head()

,@timestamp,system,hostname,loghost,logtype,envname,procname,procid,module,keywname,keywvalue,keywmask,logtext,trace_id
0,2019-04-02 01:29:45.681,MATISSE,wmt,wmt,LOG,wmt,bob_ins,21,seq,,,,Started at 2019-04-02T01:29:45 (underlined),0
1,2019-04-02 01:29:45.681,MATISSE,wmt,wmt,LOG,wmt,bob_ins,21,seq,,,,MATISSE_img_acq -- Celestial target acquisitio...,0
2,2019-04-02 01:29:45.945,MATISSE,wmt,wmt,LOG,wmt,bob_ins,21,seq,,,,DET1 INTERFY VAL = '512',0
3,2019-04-02 01:29:45.945,MATISSE,wmt,wmt,LOG,wmt,bob_ins,21,seq,,,,DET1 INTERFX VAL = '512',0
4,2019-04-02 01:29:45.945,MATISSE,wmt,wmt,LOG,wmt,bob_ins,21,seq,,,,COU AG EQUINOX = '2000.0',0


In [57]:
#Genero una funcion que saca todos los logs con cada trace id y los deja en una lista (Esta funcion es solo para utilizarla con el df_ints)
def listas_logtext(df):
    #Aca guardo los valores de las trace_id
    traces = df['trace_id'].unique()
    listas_log=[] #Lista vacia para acumular listas de logs de cada trace_id

    for traza in traces:
        trace = df[df['trace_id']== traza]
        log_text_trace_id = (list(trace['logtext']))
        listas_log.append(log_text_trace_id)
    
    return listas_log

trace_log= listas_logtext(df_inst)

In [89]:
#Este bloque de codigo es para explicar de mejor manera la funcion

print('Logs del trace_id:0')
print(f'Se encontraron {len(trace_log[0])} logs en total con trace id = 0')
print(f'Dentro de esa lista hay {len(set(trace_log[0]))} logs distintos en el trace id = 0')

#Aca solo muestro la lista de todos los logs que aparecen para el trace_id=0 (Incluyendo las repeticiones)
print('-',trace_log[0])

Logs del trace_id:0
Se encontraron 348 logs en total con trace id = 0
Dentro de esa lista hay 256 logs distintos en el trace id = 0
- ['Started at 2019-04-02T01:29:45 (underlined)', 'MATISSE_img_acq -- Celestial target acquisition (yellow)', "DET1 INTERFY VAL = '512'", "DET1 INTERFX VAL = '512'", "COU AG EQUINOX = '2000.0'", "DET1 APOY VAL = '512'", "DET1 APOTYPE VAL = '0'", "DEL REF MODE = 'AUTO'", "DEL FT SENSOR = 'MATISSE'", "COU GS MAG = '0.0'", "COU AG ALPHA = '000000.000'", "COU AG PMA = '0.0'", "DET1 APOX VAL = '512'", "DEL REF OPL = '30.0'", "COU AG TYPE = 'DEFAULT'", "DEL REF NAME = 'dl1'", "COU AG EPOCH = '2000.0'", "COU AG PMD = '0.0'", "COU AG DELTA = '000000.000'", "COU AG GSSOURCE = 'SCIENCE'", "DET1 NCOHERENT VAL = '1.0'", "DET2 SNRLIMIT VAL = '5.0'", "DET2 READ CURNAME = 'SCI-HIGH-GAIN'", "DET2 NINCOHERENT VAL = '1.0'", "DET2 APOX VAL = '512'", "DET1 READ CURNAME = 'AUTO'", "SEQ FS DET2 DIT = 'AUTO'", "SEQ CHOP ST = 'T'", "SEQ DIL WL0 = '3.5'", "SEQ FS DET1 DIT = '0.111

### Nuevo df_meta
La idea es hacer un df_meta nuevo que cada registro muestre el conjunto de logs que se generó en cada trace_id y muestre si dentro de ese conjunto de logs se presento un error en el df_meta.

In [59]:
df_nuevo_meta =  pd.DataFrame(df_inst['trace_id'].unique())
df_nuevo_meta.columns = ['trace_id']
df_nuevo_meta['trace_logs'] = trace_log
df_nuevo_meta['ERROR'] = df_meta['ERROR'].to_list()
df_nuevo_meta

,trace_id,trace_logs,ERROR
0,0,"[Started at 2019-04-02T01:29:45 (underlined), ...",False
1,1,"[Started at 2019-04-02T01:38:08 (underlined), ...",False
2,2,"[Started at 2019-04-02T01:45:44 (underlined), ...",False
3,3,"[Started at 2019-04-02T01:58:19 (underlined), ...",False
4,4,"[Started at 2019-04-02T06:36:28 (underlined), ...",False
...,...,...,...
1328,1328,"[Started at 2019-04-30T14:01:23 (underlined), ...",False
1329,1329,"[Started at 2019-04-30T22:51:00 (underlined), ...",False
1330,1330,"[Started at 2019-04-30T22:57:16 (underlined), ...",False
1331,1331,[MATISSE_hyb_obs -- Celestial target observati...,False


In [96]:
#Esta funcion es para que solo se muestren los datos en los que se genero error con el conjunto de logs que le corresponde a cada trace_id
def trace_con_errores(df_meta_actualizado):
    return df_meta_actualizado[df_meta_actualizado['ERROR']==True]

df_nuevo_meta2 = trace_con_errores(df_nuevo_meta)
df_nuevo_meta2.head()

,trace_id,trace_logs,ERROR
18,18,[MATISSE_gen_tec_check_fringes -- Fast check o...,True
33,33,[MATISSE_img_acq -- Celestial target acquisiti...,True
40,40,"[Started at 2019-04-03T03:02:03 (underlined), ...",True
62,62,[MATISSE_gen_cal_det_h2 -- Detector Characteri...,True
73,73,"[Started at 2019-04-03T15:05:25 (underlined), ...",True


In [108]:
#trazas error es la lista de logs de un trace_id en el que se genero un error
traces_trazas_error = df_nuevo_meta2['trace_id'].to_list()
trazas_error = df_nuevo_meta2['trace_logs'].to_list()

print(f'Esta es la lista del conjunto de logs que generaron al df meta un error para el trace id = {df_nuevo_meta2['trace_id'].tolist()[0]}')
print(trazas_error[0])

Esta es la lista del conjunto de logs que generaron al df meta un error para el trace id = 18
['MATISSE_gen_tec_check_fringes -- Fast check of fringes for instrumentation (yellow)', 'Started at 2019-04-02T11:52:49 (underlined)', "DET2 APOY VAL = '512'", "DET1 NDIT = '40'", "DET2 INTERFY VAL = '512'", "DET1 NINCOHERENT VAL = '1.'", "DET2 INTERFX VAL = '512'", "DET2 APOTYPE VAL = '0'", "DET2 APOX VAL = '512'", "DET1 READ CURNAME = 'SCI-FAST-SPEED'", "DET1 SNRLIMIT VAL = '5.'", "DET1 INTERFY VAL = '512'", "DET1 APOY VAL = '512'", "DET1 WLMAXCUT VAL = '4.8'", "DET2 DIT = '0.02'", "DET1 NCOHERENT VAL = '1.'", "DET1 DIT = '0.011'", "DET1 WLMINCUT VAL = '3.0'", "DET1 APOX VAL = '512'", "DET1 APOTYPE VAL = '0'", "DET1 INTERFX VAL = '512'", "DET2 WLMAXCUT VAL = '10'", "INS PIN NAME = 'PHOTO'", "INS FIN NAME = 'OPEN'", "DPR CATG = 'TECHNICAL'", "DET2 NDIT = '40'", "SEQ SIMUL DIR = '/data/MATISSE/INS_ROOT/SYSTEM/SIMDATA'", "DET2 NINCOHERENT VAL = '1.'", "DET2 WLMINCUT VAL = '8'", "INS POL NAME = 

In [86]:
print(f'Hay {len(trazas_error)} trazas que tienen error, por lo que se corrobora que no hay errores en la funcion con df_meta con error')
print(f'La primera traza con error tiene una lista de log_text con {len(trazas_error[0])} logs')
print(trazas_error[0])

Hay 42 trazas que tienen error, por lo que se corrobora que no hay errores en la funcion con df_meta con error
La primera traza con error tiene una lista de log_text con 861 logs
['MATISSE_gen_tec_check_fringes -- Fast check of fringes for instrumentation (yellow)', 'Started at 2019-04-02T11:52:49 (underlined)', "DET2 APOY VAL = '512'", "DET1 NDIT = '40'", "DET2 INTERFY VAL = '512'", "DET1 NINCOHERENT VAL = '1.'", "DET2 INTERFX VAL = '512'", "DET2 APOTYPE VAL = '0'", "DET2 APOX VAL = '512'", "DET1 READ CURNAME = 'SCI-FAST-SPEED'", "DET1 SNRLIMIT VAL = '5.'", "DET1 INTERFY VAL = '512'", "DET1 APOY VAL = '512'", "DET1 WLMAXCUT VAL = '4.8'", "DET2 DIT = '0.02'", "DET1 NCOHERENT VAL = '1.'", "DET1 DIT = '0.011'", "DET1 WLMINCUT VAL = '3.0'", "DET1 APOX VAL = '512'", "DET1 APOTYPE VAL = '0'", "DET1 INTERFX VAL = '512'", "DET2 WLMAXCUT VAL = '10'", "INS PIN NAME = 'PHOTO'", "INS FIN NAME = 'OPEN'", "DPR CATG = 'TECHNICAL'", "DET2 NDIT = '40'", "SEQ SIMUL DIR = '/data/MATISSE/INS_ROOT/SYSTEM/

In [63]:
from collections import Counter

In [109]:
#Hare una lista con conteos de los logs que se mostraron en cada trace_id
def conteo_traces_id(listas_de_logs):

    lista_logs_distintos = []
    for lista in listas_de_logs:
        conteo_elementos =Counter(lista)
        lista_logs_distintos.append(conteo_elementos)
    return lista_logs_distintos

#A la funcion de conteo de entrada le doy las listas que generaron error en los trace_id
conteo_logs = conteo_traces_id(trazas_error)

In [113]:
#Este es la frecuencia de los logs que aparecieron en el primer trac_id que tuvo error
conteo_logs[0]

Counter({'Executing SETUP command ...': 66,
         'SETUP command done.': 66,
         '1 (SpringGreen4)': 20,
         'Forward(b) SETUP to NGCIR2': 12,
         'Forward(b) SETUP to NGCIR1': 12,
         'Reply to wait cmd arrived': 12,
         'SETUP -expoId 0 -function INS.MODE AUTOTEST (blue)': 8,
         '... TPL.ID "MATISSE_gen_tec_check_fringes" TPL.NAME "Fast check of fringes for instrumentation" TPL.PRESEQ "MATISSE_gen_tec_check_fringes.seq" TPL.START "2019-04-02T11:52:49"': 8,
         'Executing START command ...': 8,
         'Send command \'EXPSTRT\' \'"NGCIR1 NGCIR2 ICS"\' to sub-system \'ICS\' ...': 8,
         "Last Reply to 'EXPSTRT' from 'ICS' received: 'OK !' (len='5')": 8,
         'Send command \'EXPEND\' \'"NGCIR1 NGCIR2 ICS"\' to sub-system \'ICS\' ...': 8,
         "Last Reply to 'EXPEND' from 'ICS' received: 'OK !' (len='5')": 8,
         '1 - ic0fbControlSrv_statusCb.C:32: Received command: STATUS': 8,
         '1 - ic0fbControlSrv_statusCb.C:79: Handled 